# 範例樣本酒店及航班預訂代理

此方案將幫助你預訂機票及酒店。情景是從倫敦希斯路機場 LHR 出發，於 2024 年 2 月 20 日飛往紐約 JFK，並於 2025 年 2 月 27 日返回，僅乘坐英國航空經濟艙。我希望在紐約入住希爾頓酒店，請提供航班及酒店的費用。


# 初始化 Azure AI Agent 服務並從 **.env** 獲取配置信息

### **.env** 

建立一個 .env 文件

**.env** 包含 Azure AI Agent 服務的連接字串、AOAI 使用的模型，以及相應的 Google API 搜索服務 API、ENDPOINT 等。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "您的 Azure AI Agent 服務模型部署名稱"

[**NOTE**] 您需要一個具有 100,000 速率限制（每分鐘標記數）且 600 速率限制（每分鐘請求數）的模型

  您可以在 Microsoft Foundry - 模型與端點中獲取模型。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "您的 Azure AI Agent 服務專案連接字串"

  您可以在 AI Foundry 入口網站畫面的專案總覽中獲取專案連接字串。

- **SERPAPI_SEARCH_API_KEY** = "您的 SERPAPI 搜索 API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "您的 SERPAPI 搜索端點"

要獲取 Azure AI Agent 服務的模型部署名稱和專案連接字串，您需要建立 Azure AI Agent 服務。建議直接使用[此範本](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)來創建（***注意：*** Azure AI Agent 服務目前設定於限制區域，建議參考[此連結](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)設定區域）

Agent 需要存取 SERPAPI，建議使用[此連結](https://serpapi.com/searches)註冊。註冊後，您可以獲得唯一的 API KEY 與 ENDPOINT。


# Setup 

要運行此筆記本，您需要確保已通過運行 `pip install -r requirements.txt` 安裝所需的庫。


In [ ]:
from semantic_kernel import __version__

__version__

你的 Semantic Kernel 版本應至少為 1.27.2。


請加載您的 .env 文件設置和資源，並請確保您已添加您的金鑰和設置，並已創建本地的 .env 文件。


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 登入 Azure

您現在需要登入 Azure。打開終端機並運行以下命令：

```bash
az login
```

此命令將提示您輸入您的 Azure 認證，使 Azure AI 代理服務能正常運作。


# Explanation:
這是一個變數，用來儲存存取 SERP（搜尋引擎結果頁面）API 服務的 API 金鑰。API 金鑰是一個用於驗證與您的帳戶相關聯請求的唯一識別碼。

Purpose: 這行的目的是將 API 金鑰儲存在變數中，以便用來驗證對 SERP API 服務的請求。存取此服務並執行搜尋時，需要用到 API 金鑰。
How to Get a SERP API Key: 若要獲取 SERP API 金鑰，請按照 https://serpapi.com 上的一般步驟進行（具體步驟可能會根據您使用的特定 SERP API 服務有所不同）：

Choose a SERP API Service: 有多個 SERP API 服務可供選擇，例如 SerpAPI、Google Custom Search JSON API 及其他。選擇最適合您需求的服務。

Sign Up for an Account: 前往您選擇的 SERP API 服務網站並註冊帳戶。您可能需要提供一些基本資訊並驗證您的電子郵件地址。

Create an API Key: 註冊後，登入您的帳戶並前往 API 區域或儀表板。尋找建立或產生新 API 金鑰的選項。
Copy the API Key to your .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: 這是一個變數，用來儲存 SERP API 端點的基本 URL。變數名稱 BASE_URL 是一個慣例，用來表示這個 URL 是進行 API 請求的起點。
'https://serpapi.com/search':

這是分配給 BASE_URL 變數的實際 URL 字串。它代表使用 SERP API 進行搜尋查詢的端點。

# Purpose:
這行的目的是定義一個常數，該常數保存 SERP API 的基本 URL。這個 URL 將用作構建 API 請求以執行搜尋操作的起點。

# Usage:
透過將基本 URL 定義在變數中，您可以在程式碼中隨時重複使用它，只要需要向 SERP API 發出請求時皆可使用。這使您的程式碼更易於維護，並減少因在多處硬編碼 URL 而導致錯誤的風險。當前的範例是 https://serpapi.com/search?engine=bing，使用的是 Bing 搜尋 API。不同的 API 可以於 https://Serpapi.com 選擇。


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# 解釋：

這是您的插件代碼所在位置。

類定義：`class BookingPlugin`：定義一個名為 BookingPlugin 的類，其中包含預訂酒店和航班的方法。

酒店預訂方法：

- `@kernel_function(description="booking hotel")`：一個描述該函數為酒店預訂核心函數的裝飾器。
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`：定義一個用於酒店預訂的方法，帶有帶註解的參數和返回類型。

該方法構造一個酒店預訂請求的參數字典，並向 SERP API 發送 GET 請求。它檢查響應狀態，若成功則返回酒店屬性，若請求失敗則返回 None。

航班預訂方法：

- `@kernel_function(description="booking flight")`：一個描述該函數為航班預訂核心函數的裝飾器。
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`：定義一個用於航班預訂的方法，帶有帶註解的參數和返回類型。

該方法構造了出發和返回航班請求的參數字典，並向 SERP API 發送 GET 請求。它檢查響應狀態，若成功則將航班信息附加到結果字串中，若請求失敗則打印錯誤訊息。該方法返回包含航班信息的結果字串。


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# 說明：
匯入敘述：匯入 Azure 憑證、AI 代理、聊天訊息內容、作者角色及核心函數裝飾器所需的模組。

非同步上下文管理：async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): 設定非同步上下文管理器以處理 Azure 憑證並建立 AI 代理客戶端。

代理名稱與指示：
- `AGENT_NAME = "BookingAgent"`：定義代理的名稱。
- `AGENT_INSTRUCTIONS = """..."""`：提供代理如何處理預訂請求的詳細指示。

建立代理定義：`agent_definition = await client.agents.create_agent(...)`：使用指定的模型、名稱和指示來建立代理定義。

建立 AzureAI 代理：`agent = AzureAIAgent(...)`：使用客戶端、代理定義和已定義的插件建立 AzureAI 代理。

建立執行緒：`thread: AzureAIAgentThread | None = None`：為代理建立一個執行緒。並不一定要先建立執行緒——若提供 `None`，首次調用時將建立新執行緒並於回應中返回。

用戶輸入：`user_inputs = ["..."]`：定義代理要處理的用戶輸入列表。

在 finally 區塊中，刪除執行緒和代理以清理資源。


# 認證

`DefaultAzureCredential` 類是 Azure SDK for Python 的一部分。它提供了一種預設方式來與 Azure 服務進行認證。它會按照特定順序嘗試使用多種方法進行認證，例如環境變數、託管身份和 Azure CLI 認證。

非同步操作：aio 模組表示 DefaultAzureCredential 類支援非同步操作。這表示您可以將它與 asyncio 一起使用，以執行非阻塞的認證請求。


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：
本文件乃使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 所翻譯。雖然我們致力於準確性，請注意自動翻譯可能包含錯誤或不準確之處。文件原文語言版本應視為權威來源。對於關鍵資訊，建議採用專業人工翻譯。本公司對因使用本翻譯所引致之任何誤解或誤釋概不負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
